In [18]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [19]:
# model = Sequential([
#     Conv2D(100, (3,3), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(2,2),
#     Conv2D(100, (3,3), activation='relu'),
#     MaxPooling2D(2,2),
#     Flatten(),
#     Dropout(0.5),
#     Dense(50, activation='relu'),
#     Dense(2, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

baseModel = MobileNetV2(include_top=False, weights='imagenet',  input_shape=(224,224,3))

headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(50, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

# compile our model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['acc'])
print("[INFO] model compiled successfully.")

9412608/9406464 [==============================] - 0s 0us/step
[INFO] model compiled successfully.


In [22]:
def get_generator(src):
    datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
        )
                                
    generator = datagen.flow_from_directory(
        src,
        target_size=(224, 224),
        batch_size=32
    )
    return generator


train_generator = get_generator('/content/Dataset/train')
validation_generator = get_generator('/content/Dataset/test')

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [23]:
checkpoint = ModelCheckpoint(
    'model2-{epoch:03d}.model',
    monitor='val_loss',verbose=0,
    save_best_only=True,
    mode='auto'
    )

In [24]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 32,
    epochs = 20,
    callbacks=[checkpoint]
    )

Epoch 1/20
41/41 [==============================] - 39s 961ms/step - loss: 1.8169 - acc: 0.8465 - val_loss: 0.0242 - val_acc: 0.9948
Epoch 2/20
41/41 [==============================] - 22s 542ms/step - loss: 0.6500 - acc: 0.9454 - val_loss: 0.2935 - val_acc: 0.9740
Epoch 3/20
41/41 [==============================] - 22s 537ms/step - loss: 0.3214 - acc: 0.9696 - val_loss: 0.1790 - val_acc: 0.9844
Epoch 4/20
41/41 [==============================] - 22s 538ms/step - loss: 0.3999 - acc: 0.9641 - val_loss: 0.0943 - val_acc: 0.9896
Epoch 5/20
41/41 [==============================] - 22s 544ms/step - loss: 0.2196 - acc: 0.9719 - val_loss: 0.0611 - val_acc: 0.9896
Epoch 6/20
41/41 [==============================] - 22s 539ms/step - loss: 0.0618 - acc: 0.9883 - val_loss: 0.0483 - val_acc: 0.9792
Epoch 7/20
41/41 [==============================] - 38s 933ms/step - loss: 0.0844 - acc: 0.9766 - val_loss: 0.0070 - val_acc: 1.0000
Epoch 8/20
41/41 [==============================] - 22s 541ms/step - 

In [25]:
model.save("/content/detector.model", save_format = "h5")